# Input Webcam

Creates video stream from webcam and forwards each frame as PNG one by one to http endpoint via POST

In [ ]:
import os
"""
os.environ['create_image']='True'
os.environ['repository']='romeokienzler'
os.environ['version']='0.14'
"""
os.environ['install_requirements']='True'

In [ ]:
if bool(os.environ.get('create_image',False)):
    docker_file="""
    FROM registry.access.redhat.com/ubi8/python-39
    RUN pip install ipython nbformat opencv-python
    ADD image-endpoint.ipynb /
    ENTRYPOINT ["ipython","/image-endpoint.ipynb","> /tmp/component.log","2> /tmp/component.err"]
    """
    with open("Dockerfile", "w") as text_file:
        text_file.write(docker_file)

    !docker build -t claimed-predict-image-endpoint:`echo $version` .
    !docker tag claimed-predict-image-endpoint:`echo $version` `echo $repository`/claimed-predict-image-endpoint:`echo $version`
    !docker push `echo $repository`/claimed-predict-image-endpoint:`echo $version`
elif bool(os.environ.get('install_requirements',False)):
    !pip install opencv-python

In [ ]:
import cv2

In [ ]:
from __future__ import print_function
import requests
import json
import cv2

vid = cv2.VideoCapture(0)

while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
    if not ret:
        print('err')
        break
  
    _, img_encoded = cv2.imencode('.png', frame)
    
    content_type = 'multipart/form-data'
    headers = {'content-type': content_type}
      
    response = requests.post('http://127.0.0.1:8080/process-labels', data=img_encoded.tostring(), headers=headers)
# decode response
    print(response.text)

In [ ]:
vid.release()


In [ ]:
vid = cv2.VideoCapture(0)
  
while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
  
    # Display the resulting frame
    cv2.imshow('frame', frame)
      
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()